In [22]:
# #pd.concat([df1[~df1.index.isin(df2.index)], df2])
# import pandas as pd
# import numpy as np

# df1 = pd.read_excel('teil1.xlsx')
# df2 = pd.read_excel('teil2.xlsx')
# df1.set_index('Transfer Order Number', inplace=True)
# df2.set_index('Transfer Order Number', inplace=True)


In [23]:
# df1 = pd.concat([df1[~df1.index.isin(df2.index)], df2])


In [24]:

import pandas as pd
import numpy as np
#from SQL import sql_datenLadenLabel,sql_datenLadenOderItems,sql_datenLadenStammdaten,sql_datenLadenOder
#from data_Class.SQL import sql_datenLadenLabel,sql_datenLadenOderItems,sql_datenLadenStammdaten,sql_datenLadenOder
    
# dfOrder = sql_datenLadenOder()
# dfOrderItems = sql_datenLadenOderItems()
# dfLabel = sql_datenLadenLabel()
# dfStammdaten = sql_datenLadenStammdaten()

In [25]:
dfStammdaten = pd.read_excel('Stammdaten.xlsx')

In [26]:
# drop from dfStammdaten all rows where in in UnitOfMeasure is not 'CS','D97' or 'OUT'



dfStammdaten = dfStammdaten[dfStammdaten['UnitOfMeasure'].isin(['CS','D97','OUT'])]       
dfStammdaten['MaterialNumber'] = dfStammdaten['MaterialNumber'].str.replace('0000000000', '')
for each in dfStammdaten.index:
    if dfStammdaten.loc[each, 'UnitOfMeasure'] == 'CS':
        dfStammdaten.loc[each, 'CS'] = dfStammdaten.loc[each, 'NumeratorToBaseUnitOfMeasure'] / dfStammdaten.loc[each, 'DenominatorToBaseUnitOfMeasure']
    else:
        dfStammdaten.loc[each, 'CS'] = 0        
    if dfStammdaten.loc[each, 'UnitOfMeasure'] == 'OUT':
        dfStammdaten.loc[each, 'OUT'] = dfStammdaten.loc[each, 'NumeratorToBaseUnitOfMeasure'] / dfStammdaten.loc[each, 'DenominatorToBaseUnitOfMeasure']
    else:
        dfStammdaten.loc[each, 'OUT'] = 0
    if dfStammdaten.loc[each, 'UnitOfMeasure'] == 'D97':
        dfStammdaten.loc[each, 'PAL'] = dfStammdaten.loc[each, 'NumeratorToBaseUnitOfMeasure'] / dfStammdaten.loc[each, 'DenominatorToBaseUnitOfMeasure']
    else:
        dfStammdaten.loc[each, 'PAL'] = 0       
#dfStammdaten.head(10)

In [27]:
dfStammdaten = pd.read_excel('Stammdaten.xlsx')

In [28]:
dfStammdaten = dfStammdaten[dfStammdaten['UnitOfMeasure'].isin(['CS','D97','OUT'])]   
def f_CS(row):
    try:
        if row.UnitOfMeasure == 'CS':          
            return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
    except:
        return np.nan
def f_PAL(row):
    try:
        if row.UnitOfMeasure == 'D97':
             return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
    except:
        return np.nan
def f_OUT(row):
    try:
        if row.UnitOfMeasure == 'OUT':
            return row.NumeratorToBaseUnitOfMeasure / row.DenominatorToBaseUnitOfMeasure
    except:
        return np.nan
dfStammdaten['OUT'] = dfStammdaten.apply(f_OUT,axis=1)
dfStammdaten['CS'] = dfStammdaten.apply(f_CS,axis=1)
dfStammdaten['PAL'] = dfStammdaten.apply(f_PAL,axis=1)
dfStammdaten.head(3)

,UnitOfMeasure,MaterialNumber,NumeratorToBaseUnitOfMeasure,DenominatorToBaseUnitOfMeasure,InternationalArticleNumber,InternationalArticleNumberCategory,Length,Width,Height,UnitOfDimension,Volume,UnitOfVolume,GrossWeight,UnitOfWeight,LowerUnitPackHierarchy,MaxStackFactor,Timestamp,OUT,CS,PAL
0,CS,000000000010002410,10,1,5.700527e+12,HE,53.3,24.6,57.8,CMT,75786.204,CMQ,15.440,KGM,NaN,0,2022-03-31 09:54:10.133,NaN,10.000,NaN
1,CS,000000000010002421,4608,1000,4.031300e+12,HE,359.0,286.0,309.0,MMT,32783.132,CMQ,6.412,KGM,NaN,0,2022-03-31 09:54:10.133,NaN,4.608,NaN
2,CS,000000000010002423,4750,1000,4.031300e+12,HE,450.0,246.0,298.0,MMT,32988.600,CMQ,6.635,KGM,NaN,0,2022-05-04 13:36:32.430,NaN,4.750,NaN


In [29]:

# dfOrderItems['MaterialNumber'] = dfOrderItems['MaterialNumber'].astype(str)
# dfOrderItems['MaterialNumber'] = dfOrderItems['MaterialNumber'].str.replace('0000000000', '')
# dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'CS'][['MaterialNumber','CS']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
# dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'D97'][['MaterialNumber','PAL']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')
# dfOrderItems = pd.merge(dfOrderItems, dfStammdaten[dfStammdaten['UnitOfMeasure'] == 'OUT'][['MaterialNumber','OUT']],left_on='MaterialNumber', right_on='MaterialNumber',how='left')

# dfOrderItems['O'] = dfOrderItems['Outers'] * dfOrderItems['OUT']
# dfOrderItems['Picks PAL'] = dfOrderItems.O / dfOrderItems.PAL
# dfOrderItems['Picks CS'] = dfOrderItems.O / dfOrderItems.CS
# dfOrderItems['Picks OUT'] = dfOrderItems.O/ dfOrderItems.OUT
# #Bereinige Berechnungen der Picks 
# for i in range(0,len(dfOrderItems.index)):
#     #----PAL bereinigen
#         if (dfOrderItems.loc[i,'Picks PAL'] <1):
#             dfOrderItems.loc[i,'Picks PAL'] = 0
#     #----cs bereinigen
#         if (dfOrderItems.loc[i,'Picks CS'] <1):
#             dfOrderItems.loc[i,'Picks CS'] = 0 
#     #mögliche PAL picks abziehen
#         if (dfOrderItems.loc[i,'Picks PAL'] >=1):
#             dfOrderItems.loc[i,'Picks CS'] = (dfOrderItems.loc[i,'O'] - (dfOrderItems.loc[i,'Picks PAL'] * dfOrderItems.loc[i,'PAL'])) * dfOrderItems.loc[i,'CS']
#     #---OUT bereinigen
#         if (dfOrderItems.loc[i,'Picks OUT'] <1):
#             dfOrderItems.loc[i,'Picks OUT'] = 0
#     #mögliche PAL picks abziehen
#         if (dfOrderItems.loc[i,'Picks PAL'] >=1):
#             dfOrderItems.loc[i,'Picks OUT'] = (dfOrderItems.loc[i,'O'] - (dfOrderItems.loc[i,'Picks PAL'] * dfOrderItems.loc[i,'PAL'])) * dfOrderItems.loc[i,'OUT']
#     #mögliche CS picks abziehen
#         if (dfOrderItems.loc[i,'Picks CS'] >=1):
#             dfOrderItems.loc[i,'Picks OUT'] = 0#(dfOrderItems.loc[i,'O'] - (dfOrderItems.loc[i,'Picks CS'] * dfOrderItems.loc[i,'CS'])) * dfOrderItems.loc[i,'OUT']
# # Picks Gesamt
# dfOrderItems['Picks Gesamt'] = dfOrderItems['Picks PAL'] + dfOrderItems['Picks CS'] + dfOrderItems['Picks OUT']
# dfOrderItems.head(5)


Wetter API 

In [35]:

# importing requests and json
import requests, json
import pandas as pd

def getWetterBayreuth():
    
    BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"
    CITY = "Bayreuth"
    API_KEY = "617894bbf9a9019b4f70056a84ddefff"
    URL = BASE_URL + "q=" + CITY + "&appid=" + API_KEY
    response = requests.get(URL)
    # add to pd dataframe
    data = response.json()
    df = pd.DataFrame(columns=['Temp','Temp Max','Temp Min','Humidity','Wind Speed','Wind Degree','Clouds','Weather'])
    df.loc[0,'Temp'] = data['main']['temp']
    df.loc[0,'Temp'] = df.loc[0,'Temp'] - 273.15
    df.loc[0,'Temp Max'] = data['main']['temp_max']
    df.loc[0,'Temp Max'] = df.loc[0,'Temp Max'] - 273.15
    df.loc[0,'Temp Min'] = data['main']['temp_min']
    df.loc[0,'Temp Min'] = df.loc[0,'Temp Min'] - 273.15
    df.loc[0,'Humidity'] = data['main']['humidity']
    df.loc[0,'Wind Speed'] = data['wind']['speed']
    df.loc[0,'Wind Degree'] = data['wind']['deg']
    df.loc[0,'Clouds'] = data['clouds']['all']
    df.loc[0,'Weather'] = data['weather'][0]['main']
    return df

# print json response
print(getWetterBayreuth())

   Temp Temp Max Temp Min Humidity Wind Speed Wind Degree Clouds Weather
0  5.41     6.39     5.01       97       5.81         270    100  Clouds
